In [ ]:
#r "nuget: Plotly.NET"
#r "nuget: Plotly.NET.Interactive"
#r "nuget: FSharp.Data"

open Plotly.NET
open FSharp.Data

In [ ]:
let round (amount : decimal) =
    Math.Round(amount, 2)
    
let getMonthlyPaymentsFromMonths (interestRate : decimal) months loan =
   float interestRate / 12. / 100.
   |> (fun rate -> rate + ( rate / ( Math.Pow(rate + 1., float months) - 1. ) ))
   |> decimal
   |> fun factor -> loan * factor |> round
   
let getMonthlyPayments interestRate years loan =
   getMonthlyPaymentsFromMonths interestRate (years * 12) loan
   
let toTable values =
    Chart.Table(headerValues = [ "Description"; "Value"; "Notes" ],
                cellsValues = values,
                CellsMultiAlign = [ StyleParam.HorizontalAlign.Left; StyleParam.HorizontalAlign.Right ])

let getLtv loan propertyValue =
    loan / propertyValue * 100m |> round
    
let getMonthlyInterest rate loan =
    loan * rate / 100m / 12m
    |> round

let getInterestMonthSeries interest years loan =
    getMonthlyPayments interest years loan
    |> (fun monthlyPayment -> List.unfold (function
                                           | (_, l) when l <= 0m      -> None
                                           | (month, outstandingLoan) -> getMonthlyInterest interest outstandingLoan
                                                                         |> (fun interest -> ((month + 1, interest), (month + 1, outstandingLoan - monthlyPayment + interest)))
                                                                         |> Some) (0, loan))

let getInterestsCost interest mortgageYears loan productYears =
    getInterestMonthSeries interest mortgageYears loan
    |> List.filter (fst >> (>=)(productYears * 12))
    |> List.sumBy snd
    
let getProductBalanceByInterest monthlyExpenses rent mortgageYears loan productYears interest =
    getInterestsCost interest mortgageYears loan productYears
    |> (+)(monthlyExpenses * 12m * decimal productYears)
    |> (-)(rent * 12m * decimal productYears)

In [ ]:
let getInterestInflectionChart currentInterest (interestData : list<decimal * decimal>) =
    let zeroNetInterest =
        interestData
        |> List.minBy (snd >> Math.Abs)
        |> fst

    let currentInterestNet =
        interestData
        |> List.find (fst >> fun interest -> currentInterest = interest)
        |> snd

    [ interestData
      |> List.map (fst >> fun interest -> interest, 0)
      |> Chart.Line
      |> Chart.withLineStyle (Dash = StyleParam.DrawingStyle.Dot,
                              Color = Color.fromKeyword ColorKeyword.Gray,
                              Width = 1.)
      |> Chart.withTitle "Net zero"
      
      interestData
      |> Chart.Line
      |> Chart.withTitle "Interests"
      
      [ zeroNetInterest, 0.0m ]
      |> Chart.Point
      |> Chart.withMarkerStyle (Size = 10, Symbol = StyleParam.MarkerSymbol.Diamond)
      |> Chart.withTitle "Inflection"
      
      [ currentInterest, currentInterestNet ]
    |> Chart.Point
    |> Chart.withMarkerStyle (Size = 10, Symbol = StyleParam.MarkerSymbol.Diamond)
    |> Chart.withTitle "Current" ]
    |> Chart.combine

##### Mortgage data

| Description       | Amount    |
| ----------------- | --------- |
| Property value    | 200000.00 |
| Loan              | 100000.00 |
| Interest          | 5.99      |
| Product years     | 29        |
| Loan duration     | 30 years  |

##### Expenses

| Description    | Amount per year code      | Reference year | Notes                                                  |
| -------------- | ------------------------- | -------------- | ------------------------------------------------------ |
| Service charge | `1000.00m * 2m`           | 2023           | 1000 twice a year                                      |
| Ground rent    | `300m`                    | 2023           |                                                        |
| Insurance      | `200.00m`                 | 2023           |                                                        |
| Agency fees    | `1000m * 12m * 5m / 100m` | 2022           | Agency takes 5% of the annual rent (£1000 * 12mm * 5%) |

In [ ]:
let mortgage =
    {| PropertyValue = 200000.00m
       Loan          = 100000.00m
       Years         = 30
       Interest      = 5.99m
       ProductYears  = 2 |}

let rent =
    1000m

let expenses = 
    {| ServiceCharge = 1000.00m * 2m
       GroundRent    = 300m
       Insurance     = 200.00m
       AgencyFees    = rent * 12m * 8m / 100m |}

In [ ]:
let totalExpenses =
    (expenses.AgencyFees + expenses.GroundRent + expenses.ServiceCharge + expenses.Insurance) / 12m
    |> round

let monthlyPayments =
    getMonthlyPayments mortgage.Interest mortgage.Years mortgage.Loan

let monthlyInterest =
    getMonthlyInterest mortgage.Interest mortgage.Loan

let netRepayments =
    monthlyPayments - monthlyInterest

let netGain =
    rent - totalExpenses - monthlyInterest

let ltv =
    getLtv mortgage.Loan mortgage.PropertyValue

[ [ "LTV"              ; $"{ltv}"            ; "Lower is better"        ]
  [ ""                 ; ""                  ; ""                       ]
  [ "Mortgage payments"; $"{monthlyPayments}"; ""                       ]
  [ "Interest"         ; $"{monthlyInterest}"; ""                       ]
  [ "Debt deduction"   ; $"{netRepayments}"  ; ""                       ]
  [ ""                 ; ""                  ; ""                       ]
  [ "Rent"             ; $"{rent:f2}"        ; ""                       ]
  [ "Interest"         ; $"{monthlyInterest}"; ""                       ]
  [ "Expenses"         ; $"{totalExpenses}"  ; "Monthly"                ]
  [ "Net"              ; $"{netGain}"        ; "Rent-interest-expenses" ]
  [ ""                 ; ""                  ; ""                       ]
  [ "Net yearly"       ; $"{netGain * 12m}"  ; "Net * 12"               ] ]
|> toTable
|> display

In [ ]:
[ 5.50m .. 0.01m .. 11.00m ]
|> List.map (fun interest -> interest, getMonthlyInterest interest mortgage.Loan |> (-) (rent - totalExpenses))
|> getInterestInflectionChart mortgage.Interest

In [ ]:
getInterestMonthSeries mortgage.Interest mortgage.Years mortgage.Loan
|> Chart.Line
|> Chart.withTitle "Interests over time (months elapsed)"

In [ ]:
let totalInterestsCost =
    getInterestsCost mortgage.Interest mortgage.Years mortgage.Loan mortgage.ProductYears

let expensesDuringProduct =
    totalExpenses * 12m * decimal mortgage.ProductYears

let totalCost =
    totalInterestsCost + expensesDuringProduct
    
let totalRevenue =
    rent * 12m * decimal mortgage.ProductYears

let mortgageNet =
    totalRevenue - totalCost
    
[ [ "Total interests cost"; $"{totalInterestsCost}"   ; ""                   ]
  [ "Total expenses"      ; $"{expensesDuringProduct}"; ""                   ]
  [ ""                    ; ""                        ; ""                   ]
  [ "Total costs"         ; $"{totalCost}"            ; "Interests+expenses" ]
  [ ""                    ; ""                        ; ""                   ]
  [ "Total revenue"       ; $"{totalRevenue}"         ; "Rent"               ]
  [ ""                    ; ""                        ; ""                   ]
  [ "Net"                 ; $"{mortgageNet}"          ; ""                   ] ]
|> toTable
|> Chart.withTitle $"Income and outcome over the {mortgage.ProductYears} years of the mortgage product"


In [ ]:
[ 5.50m .. 0.01m .. 11.00m ]
|> List.map (fun interest -> (interest, getProductBalanceByInterest totalExpenses rent mortgage.Years mortgage.Loan mortgage.ProductYears interest))
|> getInterestInflectionChart mortgage.Interest